# PPG analysis

In [ ]:
# Automatically reload modules
# %load_ext autoreload
# %autoreload 2

import os
from paradigma.config import PPGConfig, IMUConfig, SignalQualityFeatureExtractionConfig, SignalQualityClassificationConfig, HeartRateExtractionConfig
from paradigma.ppg_preprocessing import scan_and_sync_segments, preprocess_ppg_data
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features, signal_quality_classification

In [ ]:
# Cell has the tag 'parameters'
branch = 'heart_rate'
sensor = 'ppg'

path_to_data =  '../../../tests/data'
path_to_classifier = os.path.join(path_to_data, '0.classification', sensor)
path_to_sensor_data = os.path.join(path_to_data, '1.sensor_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', sensor)
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', sensor)
path_to_signal_quality = os.path.join(path_to_data, '4.predictions', sensor)
path_to_hr_estimate = os.path.join(path_to_data, '5.quantification', sensor)

In [ ]:
ppg_config = PPGConfig()
imu_config = IMUConfig()

metadatas_ppg, metadatas_imu = scan_and_sync_segments(
    os.path.join(path_to_sensor_data, 'ppg'),
    os.path.join(path_to_sensor_data, 'imu')
)

df_ppg_proc, df_imu_proc = preprocess_ppg_data(
    metadatas_ppg[0], 
    metadatas_imu[0],
    path_to_preprocessed_data,
    ppg_config, 
    imu_config
)

In [ ]:
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features

config = SignalQualityFeatureExtractionConfig()
df_features = extract_signal_quality_features(df_ppg_proc, config)

print(df_features.shape)
df_features.head()

In [ ]:
# NUMPY
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features_numpy

config = SignalQualityFeatureExtractionConfig()
df_features_numpy = extract_signal_quality_features_numpy(df_ppg_proc, config)

print(df_features_numpy.shape)
df_features_numpy.head()

In [ ]:
config = SignalQualityFeatureExtractionConfig()
df_windowed = extract_signal_quality_features(df_ppg_proc, config)

In [ ]:
config = SignalQualityClassificationConfig()
df = signal_quality_classification(df_windowed, config, path_to_classifier)